In [ ]:
import os, types, sys
from pathlib import Path
from app.utils.agent_factory import build_agent
from app.registries.system_registry import SYSTEM_REGISTRY
from app.utils.symbol_graph.symbol_service import SymbolService

# Patch engine to avoid OpenAI dependency
class FakeEngine:
    def call_engine(self, prompt: str, config: dict):
        return 'fake response'

fake_module = types.SimpleNamespace(OpenAiGpt4oAgentEngineRunner=FakeEngine)
sys.modules['app.utils.engine.openai_gpt4o_runner'] = fake_module
os.environ['OPENAI_API_KEY'] = 'fake'

symbol_service = SymbolService(Path('.'))
agent = build_agent('generator', symbol_service)
result = agent.run({'symbol': 'x', 'file': __file__, 'experiment_id': 'demo', 'run_id': '1', 'round': 1, 'system': 'demo', 'log_path': Path('.')})
print('Result:', result['response'])
print('System config:', SYSTEM_REGISTRY['refactoring'])